In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.models import Span
from shapely.geometry import Point
import geopandas as gpd
import glob
from datetime import datetime
from bokeh.layouts import Row, column, gridplot
from bokeh.models import Title, Legend, TapTool, Range1d
import matplotlib as mpl
output_notebook()


c:\Users\sahit\anaconda3\envs\datapartnership\lib\site-packages\geopandas\_compat.py:111: UserWarning: The Shapely GEOS version (3.10.1-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


Loading BokehJS ...

In [2]:
color_palette = [  '#4E79A7',  # Blue
    '#F28E2B',  # Orange
    '#E15759',  # Red
    '#76B7B2',  # Teal
    '#59A14F',  # Green
    '#EDC948',  # Yellow
    '#B07AA1',  # Purple
    '#FF9DA7',  # Pink
    '#9C755F',  # Brown
    '#BAB0AC',  # Gray
    '#7C7C7C',  # Dark gray
    '#6B4C9A',  # Violet
    '#D55E00',  # Orange-red
    '#CC61B0',  # Magenta
    '#0072B2',  # Bright blue
    '#329262',  # Peacock green
    '#9E5B5A',  # Brick red
    '#636363',  # Medium gray
    '#CD9C00',  # Gold
    '#5D69B1',  # Medium blue
]

# Meta Mobile Network Connectivity

In [25]:
all_files = glob.glob('../data/network_coverage/network_undetected/*.csv')

networkCoverage = pd.DataFrame()

li = []

for file in all_files:
    #print(file.split('_')[3])
    df1 = pd.read_csv(file)
    df1['date'] = datetime.strptime(file.split('_')[3], '%Y-%m-%d')
    li.append(df1)

networkCoverage = pd.concat(li, axis=0)
networkCoverage = networkCoverage[networkCoverage['country']=='TR']

In [4]:
turkey_adm2 = gpd.read_file('../data/turkey_administrativelevels0_1_2/tur_polbna_adm2.shp')

In [5]:
def clean_df(networkCoverage, date=None):
    if date is not None:
        df = networkCoverage[networkCoverage['date']==date]
    else:
        df = networkCoverage
    df = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.lon, df.lat))
    df = turkey_adm2.sjoin(df, how = 'left')
    df = df.groupby(['adm2_en', 'date']).sum().reset_index()[['adm2_en', 'date', 'no_coverage']]
    df = df.merge(turkey_adm2[['adm2_en', 'adm1_en']], on = ['adm2_en'], how = 'right').sort_values(by=['adm2_en', 'date']).reset_index(drop=True)

    return df

## Get Changes in Network Connectivity

In [26]:
networkCoverage = clean_df(networkCoverage)
#calculate sum of values, grouped by week
week = networkCoverage.groupby([pd.Grouper(key='date', freq='W-MON'), 'adm2_en', 'adm1_en']).mean().reset_index()

c:\Users\sahit\anaconda3\envs\datapartnership\lib\site-packages\geopandas\geodataframe.py:1938: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


In [7]:
aoi = gpd.read_file('../data/SYRTUR_tessellation.geojson')['ADM2_PCODE'].unique()
aoi_adm1 = turkey_adm2[turkey_adm2['pcode'].isin(aoi)]['adm1_en']

In [16]:
title = "Network Connectivity Trend"#"Business Activity Trends per Business Vertical in February 2022 (Compared to Pre-COVID Baseline)"

p2 = figure(title = title, x_axis_type = 'datetime', width = 800, height = 400, toolbar_location='above')
p2.add_layout(Legend(), "right")

affected_regions = ['GAZIANTEP', 'HATAY', 'KAHRAMANMARAS']

for id, adm1 in enumerate(turkey_adm2['adm1_en'].unique()):
    df = networkCoverage[networkCoverage['adm1_en']==adm1][['date', 'no_coverage']].groupby('date').mean().reset_index().sort_values(by='date')
    if adm1 not in (['GAZIANTEP', 'HATAY', 'KAHRAMANMARAS']):
        p2.line(df['date'], df['no_coverage'], line_width=2, line_color = '#d2d2d2')

for id, adm1 in enumerate(affected_regions):
    df = networkCoverage[networkCoverage['adm1_en']==adm1][['date', 'no_coverage']].groupby('date').mean().reset_index().sort_values(by='date')
    p2.line(df['date'], df['no_coverage'], line_width=4, line_color = color_palette[affected_regions.index(adm1)], legend_label=adm1)

p2.renderers.extend([
        Span(
            location=datetime(2023, 2, 6),
            dimension="height",
            line_color='#7C7C7C',
            line_width=1,
            line_dash=(4,4)
      ),
        Span(
            location=datetime(2023, 2, 20),
            dimension="height",
            line_color='#7C7C7C',
            line_width=1,
            line_dash=(4,4)
        ),
    ]
)

p2.yaxis.minor_tick_line_color = None
#p2.y_range = Range1d(0, 0.8)
p2.legend.click_policy='hide'


title_fig = figure(title='Mobile Network Connectivity (compared to 30 day prior baseline)', toolbar_location=None,width=1200, height=50, )
title_fig.title.align = "left"
title_fig.title.text_font_size = "20pt"
title_fig.border_fill_alpha = 0
title_fig.outline_line_width=0
tap = TapTool(behavior="inspect")
p2.add_tools(tap)

sub_title = figure(title='Source: Data for Good at Meta', toolbar_location=None,width=1000, height=50, )
sub_title.title.align = "left"
sub_title.title.text_font_size = "10pt"
sub_title.title.text_font_style="normal"
sub_title.border_fill_alpha = 0
sub_title.outline_line_width=0


row = Row(children=[p2])
grid = gridplot([[row]])
layout = column(title_fig, grid, sub_title)

show(layout)

In [29]:
title = "Network Connectivity Trend"#"Business Activity Trends per Business Vertical in February 2022 (Compared to Pre-COVID Baseline)"

p2 = figure(title = title, x_axis_type = 'datetime', width = 800, height = 400, toolbar_location='above')
p2.add_layout(Legend(), "right")

affected_regions = ['GAZIANTEP', 'HATAY', 'KAHRAMANMARAS']

for id, adm1 in enumerate(turkey_adm2['adm1_en'].unique()):
    df = week[week['adm1_en']==adm1][['date', 'no_coverage']].groupby('date').mean().reset_index().sort_values(by='date')
    if adm1 not in (['GAZIANTEP', 'HATAY', 'KAHRAMANMARAS']):
        p2.line(df['date'], df['no_coverage'], line_width=2, line_color = '#d2d2d2')

for id, adm1 in enumerate(affected_regions):
    df = week[week['adm1_en']==adm1][['date', 'no_coverage']].groupby('date').mean().reset_index().sort_values(by='date')
    p2.line(df['date'], df['no_coverage'], line_width=4, line_color = color_palette[affected_regions.index(adm1)], legend_label=adm1)

p2.renderers.extend([
        Span(
            location=datetime(2023, 2, 6),
            dimension="height",
            line_color='#7C7C7C',
            line_width=1,
            line_dash=(4,4)
      ),
        Span(
            location=datetime(2023, 2, 20),
            dimension="height",
            line_color='#7C7C7C',
            line_width=1,
            line_dash=(4,4)
        ),
    ]
)

p2.yaxis.minor_tick_line_color = None
#p2.y_range = Range1d(0, 0.8)
p2.legend.click_policy='hide'


title_fig = figure(title='Mobile Network Connectivity (Weekly average compared to 30 day prior baseline)', toolbar_location=None,width=1200, height=50, )
title_fig.title.align = "left"
title_fig.title.text_font_size = "20pt"
title_fig.border_fill_alpha = 0
title_fig.outline_line_width=0
tap = TapTool(behavior="inspect")
p2.add_tools(tap)

sub_title = figure(title='Source: Data for Good at Meta', toolbar_location=None,width=1000, height=50, )
sub_title.title.align = "left"
sub_title.title.text_font_size = "10pt"
sub_title.title.text_font_style="normal"
sub_title.border_fill_alpha = 0
sub_title.outline_line_width=0


row = Row(children=[p2])
grid = gridplot([[row]])
layout = column(title_fig, grid, sub_title)

show(layout)

### Writing files to csv

In [ ]:
for date in ['2023-02-13', '2023-02-20', '2023-02-27', '2023-03-06']:
        
    df = week[week['week_date']==date][['week_date','adm2_en', 'no_coverage']].merge(turkey_adm2[['adm2_en', 'adm1_en']],on='adm2_en', how = 'right').reset_index(drop=True)
    df['week_date'] = date
    df.sort_values(by='adm2_en').to_csv(f'../data/network_coverage/final/no_network_coverage_{date}.csv')